<a href="https://colab.research.google.com/github/alexeiveselov92/Yandex-Data-Analysis/blob/master/1.%20%D0%98%D1%81%D1%81%D0%BB%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5%20%D0%BD%D0%B0%D0%B4%D0%B5%D0%B6%D0%BD%D0%BE%D1%81%D1%82%D0%B8%20%D0%B7%D0%B0%D1%91%D0%BC%D1%89%D0%B8%D0%BA%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Загрузка библиотек


In [0]:
import pandas as pd
import math as mth
from scipy import stats as st
import random
import numpy as np

#визуализация
import matplotlib.pyplot as plt
from plotly import graph_objects as go
import plotly.express as px
import seaborn as sns; sns.set()
import plotly
plotly.io.renderers.default = 'colab'

#другое
pd.options.display.expand_frame_repr = False
import warnings
warnings.filterwarnings('ignore')

#загрузка файлов с диска
from google.colab import drive
drive.mount('/content/drive')

#ML
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Загрузка датасета и предобработка данных

### Пропуски в данных

In [0]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/datasets/1_credit_history.csv')
df.info()
gaps_in_the_data = pd.DataFrame({'percent_of_nulls':df.isnull().sum()/len(df)})
gaps_in_the_data['percent_of_nulls'] = gaps_in_the_data['percent_of_nulls'].map(lambda x: '{:.2%}'.format(x))
print('')
print(df.query('days_employed.isnull()', engine='python')[['days_employed', 'total_income']].sum())
gaps_in_the_data.sort_values(by='percent_of_nulls', ascending=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB

days_employed    0.0
total_income     0.0
dtype: float64


,percent_of_nulls
days_employed,10.10%
total_income,10.10%
children,0.00%
dob_years,0.00%
education,0.00%
education_id,0.00%
family_status,0.00%
family_status_id,0.00%
gender,0.00%
income_type,0.00%


> Мы видим в столбцах `days_employed` и `total_income` пропущенные значения, причём они находятся в одних и тех же строках.




### Обработка пропусков

In [0]:
#найдем ср. знач и медиану дохода и стажа соотв.
total_income_median = df['total_income'].median()
days_employed_mean = df['days_employed'].mean()

#заменим отрицательные числа на положительные в стаже
def negative_to_pozitive(value):
    if value < 0:
        value *= -1
        return value
    else:
        return value
df['days_employed'] = df['days_employed'].apply(negative_to_pozitive)
df['days_employed'] = df['days_employed'].fillna(days_employed_mean)
df['total_income'] = df['total_income'].fillna(total_income_median)

> В стаже отрицательные числа сделали положительными, пропуски заменили на среднее значение. В доходе пропуски заменили на медианное значение.
Обратили внимание на неадекватность стажа у пенсионеров, стоят завышенные значения, в 25% выборки. При использовании этих значений в расчетах, мы бы получили неправльную картину. О причинах неадекватности значений надо было бы уточнить у операторов и исправить ошибку.

### Замена типа данных

In [0]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

> В стаже и доходе стояли значения типа float - заменили их на int.

### Обработка дубликатов

In [0]:
from collections import Counter
#приведем в единый формат столбец образование (уберем дубликаты, полученные из-за разного регистра букв)
Counter(df['education'])
df['education'] = df['education'].str.lower()
Counter(df['education'])

#проверим формат столбца семейный статус
df
Counter(df['family_status'])
df.loc[df.loc[:, 'gender'] == 'XNA']

#проверим формат столбца профессии
Counter(df['income_type'])

#проверим формат столбца цель кредита
Counter(df['purpose'])

#проверим формат столбца количества детей
df['children'].value_counts()
#видим ошибку - присутствует значение -1, а также 20 в столбце количества детей
# -1 заменим на 1, а 20 заменим медианным значением, так как 
# мы не знаем как возникла ошибка: случайно ввели лишний ноль при вводе 2 или же вместо 0 случайно поставили 20
df['children'] = df['children'].replace(-1, 1)
children_median = df.loc[df.loc[:, 'children'] != 20]['children'].median()
df['children'] = df['children'].replace(20, children_median)
pd.DataFrame({'Кол-во заёмщиков по кол-ву детей':df['children'].value_counts()})

,Кол-во заёмщиков по кол-ву детей
0,14225
1,4865
2,2055
3,330
4,41
5,9


> Привели значения образования в единый формат, проверили остальные столбцы через метод Counter и метод value_counts(). Нашлась 1 строка со значением пола - 'XNA' - скорее всего пропуск, но так как мы не знаем пол человека и не можем сделать предположений из данных, а также в расчетах использовать значение не будем, то оставляем его как есть. Также были отрицательные значения в количестве детей и неадекватное значение 20 детей. Все их обработали.

### Категоризация

In [0]:
#столбец цель кредита нужно лемматизировать и на основе его создать новый столбец с категориями 

estate = ['недвижим', 'жиль']
wedding = ['свадьб']
auto = ['автомоб']
education = ['образова']

df['purpose_category'] = 0
def categoryzer(list_of_words, category):
  join = '|'.join(list_of_words)
  index = df[df['purpose'].str.lower().str.contains(join)].index.to_list()
  for i in index:
    df.loc[i, 'purpose_category'] = category
  return df


categoryzer(wedding, 'свадьба')
categoryzer(estate, 'недвижимость')
categoryzer(auto, 'авто')
categoryzer(education, 'образование')

#проверим результат и убедимся, что значений None не осталось
Counter(df['purpose_category'])

Counter({'авто': 4315,
         'недвижимость': 10840,
         'образование': 4022,
         'свадьба': 2348})

> Видим, что многие уникальные значения цели кредита названы похожим образом, но имеют одинаковый смысл. Мы лемматизируем их, чтобы затем разбить на категории. Обе задачи мы выполнили с помощью одной функции. В итоге мы получили 4 категории целей кредита.
В нашей задаче это не понадобится, но наша категоризация сможет помочь другим аналитикам, которые изучают влияние цели кредита на возврат денег в срок.

* 'недвижимость': 10840,
* 'авто': 4315,
* 'образование': 4022,
* 'свадьба': 2348

In [0]:
debt_from_children = pd.DataFrame()
debt_from_children['count_children'] = df.groupby('children')['debt'].count()
debt_from_children['sum_children'] = df.groupby('children')['debt'].sum()
debt_from_children['result_children'] = debt_from_children['sum_children'] / debt_from_children['count_children'] 
debt_from_children

,count_children,sum_children,result_children
children,,,
0,14225,1071,0.075290
1,4865,445,0.091470
2,2055,194,0.094404
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


> Так как мы изучаем влияние семейного положения и наличия детей на факт погашения в срок, мы не будем разбивать возраста на категории, так как они нам не понадобятся. Для сдачи проекта категоризация была применена вместе с лемматизацией выше в одной функции. Категоризовать количество детей нет необходимости, категориями будет как раз их количество, так как всего 6 вариантов.

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [0]:
debt_from_children = pd.DataFrame()
debt_from_children['count_children'] = df.groupby('children')['debt'].count()
debt_from_children['sum_children'] = df.groupby('children')['debt'].sum()
debt_from_children['result_children'] = debt_from_children['sum_children'] / debt_from_children['count_children'] 
debt_from_children.sort_values('result_children', ascending = False)

,count_children,sum_children,result_children
children,,,
4,41,4,0.097561
2,2055,194,0.094404
1,4865,445,0.091470
3,330,27,0.081818
0,14225,1071,0.075290
5,9,0,0.000000


> С увеличением количества детей мы видим увеличение количества просроченных задолженностей, хотя люди с 3 детьми чаще платят в срок чем люди с 1 ребенком. Данные не однозначные, возможно, нужна большая выборка, чем та, которую мы имеем. 
Бездетные, как правило реже просрачивают оплату по кредиту, чем люди с детьми.

### Есть ли зависимость между семейным статусом и возвратом кредита в срок?

In [0]:
debt_from_family_status = pd.DataFrame()
debt_from_family_status['sum_family_status'] = df.groupby('family_status')['debt'].sum()
debt_from_family_status['count_family_status'] = df.groupby('family_status')['debt'].count()
debt_from_family_status['result_family_status'] = debt_from_family_status['sum_family_status'] / debt_from_family_status['count_family_status'] 
debt_from_family_status.sort_values('result_family_status', ascending = False)

,sum_family_status,count_family_status,result_family_status
family_status,,,
Не женат / не замужем,274,2813,0.097405
гражданский брак,388,4177,0.092890
женат / замужем,931,12380,0.075202
в разводе,85,1195,0.071130
вдовец / вдова,63,960,0.065625


> Да, зависимость есть. Люди не в браке и не бывавшие в браке имеют больший процент невозвратов в срок. 
Но, те кто развелись или овдовели чаще платят в срок, чем люди в браке.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [0]:
def total_income_category(value):
    if value >= 500000:
        return 10
    else:
        value = value // 50000
        return value
df['total_income_id'] = df['total_income'].apply(total_income_category)
df['total_income_id'].value_counts()
debt_from_total_income = pd.DataFrame()
debt_from_total_income['sum'] = df.groupby('total_income_id')['debt'].sum()
debt_from_total_income['count'] = df.groupby('total_income_id')['debt'].count()
debt_from_total_income['conversion'] = debt_from_total_income['sum'] / debt_from_total_income['count']
debt_from_total_income.sort_values('conversion', ascending = False)

#Ранее мы категоризировали уровень дохода создав доп столбец total_income_id. Мы выполнили целочисленное 
#деление дохода на 50000, тем самым разбив люей на категории каждые 50000 дохода. 
#Мы видим, что начиная с id 10 (зп >= 500000) выборка людей становится нерепрезентативной, поэтому объединяем
#людей с доходом => 500000 в одну категорию - id = 10. Для этого добавим в функцию условие.

#видим что зависимости долга от доходов кадется нет, на всякий случай посмотрим корреляцию между столбцами дохода и долгом
verification_through_correlation = pd.DataFrame()
verification_through_correlation['total_income'] = df['total_income']
verification_through_correlation['debt'] = df['debt']
verification_through_correlation.corr()
#видим, что корреляции нет, теперь вывод очевиден

,total_income,debt
total_income,1.000000,-0.011592
debt,-0.011592,1.000000


> Зависимости между уровнем дохода и возвратом кредита в срок нет!

### Как разные цели кредита влияют на его возврат в срок?

In [0]:
debt_from_purpose_category = pd.DataFrame()
debt_from_purpose_category['sum_purpose_category'] = df.groupby('purpose_category')['debt'].sum()
debt_from_purpose_category['count_purpose_category'] = df.groupby('purpose_category')['debt'].count()
debt_from_purpose_category['result_purpose_category'] = debt_from_purpose_category['sum_purpose_category'] / debt_from_purpose_category['count_purpose_category'] 
debt_from_purpose_category.sort_values('result_purpose_category', ascending = False)

,sum_purpose_category,count_purpose_category,result_purpose_category
purpose_category,,,
авто,403,4315,0.093395
образование,370,4022,0.091994
свадьба,186,2348,0.079216
недвижимость,782,10840,0.072140


> Вот нам и понадобилось разбиение целей кредита по категориям.
Как мы видим, среди 4 наших категорий, чаще всего платят в срок с целью кредита недвижимость, чаще просрочка у тех кому нужен кредит на авто и образование. Кредиты на свадьбу просрачивают реже - примерно как и в случае недвижимости.

## Общий вывод

Общий вывод - семейный статус влияет на вероятность платежей по кредиту в срок - люди, бывший в браке чаще плятят в срок чем те, кто не бывали в браке, причем разведенные и овдовевшие платят в срок чаще, чем люди в браке. Чем больше детей, тем чаще платят не в срок, бездетные реже просрачивают оплату.